![Astrofisica Computacional](../logo.png)

# Example: Searching for Neighbor Points

Eduard Larrañaga (ealarranaga@unal.edu.co)

In [19]:
import numpy as np
#import sys

rng = np.random.default_rng(413)  # initialise our random number generator

We will create an array with the 3D-coordinates, randomly generated in the range \[0,1), of  5 points:

In [32]:
N = 4
dimension = 3
points = rng.random((N, dimension))
points

array([[0.59527896, 0.72462972, 0.93176844],
       [0.67188895, 0.06276041, 0.06577559],
       [0.50963023, 0.97577275, 0.29005854],
       [0.94299795, 0.46371326, 0.15239736]])

In [33]:
points[1] # Coordinates of one of the points

array([0.67188895, 0.06276041, 0.06577559])

We want to find the nearest point for each one, assuming an Euclidean distance. We will use the function `numpy.linalg.norm` to calculate the distances.

Info: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.

The norm of the position of a single point is 

In [34]:
np.linalg.norm(points[1])

0.6780118409192759

The norm of all the points is given by

In [35]:
np.linalg.norm(points, axis=1)

array([1.32198249, 0.67801184, 1.1384153 , 1.06183807])

Now we reshape the array of coordinates by introducing another dimension:

In [37]:
resh_points = points.reshape(N, 1, dimension)
resh_points

array([[[0.59527896, 0.72462972, 0.93176844]],

       [[0.67188895, 0.06276041, 0.06577559]],

       [[0.50963023, 0.97577275, 0.29005854]],

       [[0.94299795, 0.46371326, 0.15239736]]])

This let us calculate the matrix of differences of coordinates

In [39]:
Delta_coordinates = resh_points - points
Delta_coordinates

array([[[ 0.        ,  0.        ,  0.        ],
        [-0.07660999,  0.66186931,  0.86599285],
        [ 0.08564873, -0.25114303,  0.6417099 ],
        [-0.347719  ,  0.26091646,  0.77937108]],

       [[ 0.07660999, -0.66186931, -0.86599285],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.16225872, -0.91301233, -0.22428295],
        [-0.27110901, -0.40095284, -0.08662176]],

       [[-0.08564873,  0.25114303, -0.6417099 ],
        [-0.16225872,  0.91301233,  0.22428295],
        [ 0.        ,  0.        ,  0.        ],
        [-0.43336772,  0.51205949,  0.13766119]],

       [[ 0.347719  , -0.26091646, -0.77937108],
        [ 0.27110901,  0.40095284,  0.08662176],
        [ 0.43336772, -0.51205949, -0.13766119],
        [ 0.        ,  0.        ,  0.        ]]])

In [46]:
# Note that the shapes change as
print(resh_points.shape, " - ", points.shape, " => ", Delta_coordinates.shape)

(4, 1, 3)  -  (4, 3)  =>  (4, 4, 3)


Now, we use the `numpy.linalg.norm()` function to calculate the matrix of distances:

In [40]:
distances = np.linalg.norm((points.reshape(N, 1, n_dims) - points), axis=2)
distances

array([[0.        , 1.09264984, 0.69440631, 0.89241537],
       [1.09264984, 0.        , 0.95405569, 0.49169768],
       [0.69440631, 0.95405569, 0.        , 0.68480881],
       [0.89241537, 0.49169768, 0.68480881, 0.        ]])

It is clear that the diagonal contains the information of the "self-distance". We correct these values in order to obtain the minimum values,

In [41]:
distances[np.diag_indices_from(distances)] = np.inf
distances

array([[       inf, 1.09264984, 0.69440631, 0.89241537],
       [1.09264984,        inf, 0.95405569, 0.49169768],
       [0.69440631, 0.95405569,        inf, 0.68480881],
       [0.89241537, 0.49169768, 0.68480881,        inf]])

Finally, we find the minimum values for each column, giving the nearest point list:

In [42]:
np.argmin(distances, axis=1)

array([2, 3, 3, 1])

The above solution is summarised in the following three steps

In [47]:
distances = np.linalg.norm((points.reshape(N, 1, n_dims) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
nearest_points = np.argmin(distances, axis=1)
nearest_points

array([2, 3, 3, 1])

### Comparing to `scikit-learn`

Now we will use the built-in function `sklearn.neighbors.NearestNeighbors`.

In [49]:
from sklearn.neighbors import NearestNeighbors
_, indices = NearestNeighbors().fit(points).kneighbors(points, 2)
indices[:, 1]

array([2, 3, 3, 1])

Lets measure the time to find the nearest points using both methods.

In [50]:
%timeit _, indices = NearestNeighbors().fit(points).kneighbors(points, 2)

81.8 µs ± 492 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [51]:
%%timeit
distances = np.linalg.norm((points.reshape(N, 1, n_dims) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
np.argmin(distances, axis=1)

10.8 µs ± 9.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now we can increase the number of points...

In [52]:
N = 1000
dimension = 3
points = rng.random((N, dimension))
points

array([[0.80135557, 0.09970322, 0.96482296],
       [0.31828742, 0.36930542, 0.54105854],
       [0.39276773, 0.62638356, 0.66135177],
       ...,
       [0.38051883, 0.33369691, 0.73145435],
       [0.9158936 , 0.23726392, 0.78531068],
       [0.34296009, 0.86598157, 0.00134824]])

In [54]:
%%timeit
distances = np.linalg.norm((points.reshape(N, 1, n_dims) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
np.argmin(distances, axis=1)

20.7 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
%timeit _, indices = NearestNeighbors().fit(points).kneighbors(points, 2)

864 µs ± 6.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
